### Попробуем здесь адаптировать новые формулы факторизации для модели Б-Ш, чтобы сравнить их впоследствии с имеющимися.

In [11]:
from numpy import log, sqrt
from numpy import exp, linspace, fft, array, arange, pi
import matplotlib.pyplot as plt
i = complex(0, 1)

Параметры контракта down-and-out put

In [12]:
# model parameters - we are going to evaluate a barrier put
T = 1
H_original = 90.0 # limit
K_original = 100.0 # strike
r_premia = 10 # annual interest rate
r = log(r_premia/100 + 1)
V0 = 0.316227766


In [18]:
# Определим функцию выплат:
def G(x, barrier, strike):
    """payoff_function for a given option type (down-and-out put there)"""
    if (barrier <= x) and (x <= strike):
        return K_original - H_original*exp(x)
    else:
        return 0

In [ ]:
# И функцию-индикатор:

def indicator(a, barrier):
    """the indicator influences the function argument, not value.
    So here it iterates through x-domain and cuts any
    values of function with an argument less than H"""
    indicated_array = []
    for index in range(len(x_space)):
        if x_space[index] > barrier:
            indicated.append(a[index])
        else:
            indicated.append(0)
    return indicated_array 

Характеристическая экспонента процесса Б-Ш
$$\psi(\xi) = \frac{{\color{green}{\sigma}}^2}{2} \xi^2 - i{\color{green}{\gamma}}\xi$$

In [21]:
sigma = V0
gamma = r - 0.5 * sigma**2 # Black-Scholes parameter, from a no-arbitrage condition

def psi(xi):
    return (sigma^2/2) * xi^2 - i*gamma*xi

Подготавливаем дискретизацию, задаём шкалу по переменной, связанной с ценой, производим замены
$$ y = ln(S)$$
$$ K = ln(K), H = ln(H) $$

Укладываем $M$ точек между $L \cdot ln(1/2)$  и $ L \cdot ln(2) $

In [14]:
# constants in log scale for price, by Zanette
M = 2**12  # number of points in price grid
L = 2  # scaling coefficient
y_min = L * log(0.5)
y_max = L * log(2.0)
d = (y_max - y_min) / M  # discretization step
K = log(K_original/H_original)
H = log(H_original/H_original)
y_space = []  # prices array
for p_elem_index in range(M):
    y_space.append(y_min + p_elem_index * d)

Теперь дисктеретизация по времени:

In [15]:
original_prices_array = H_original * exp(y_space)
# time discretization
N = 50  # number_of_time_steps
delta_t = T/N

Величины, связанные с факторизацией. В этой модели всё просто - при постоянной вариации нет необходимости учитывать эффекты решётки волатильности.

Действие, которое мы хотим произвести на каждом шаге по времени, выглядит так:
$$f_n(y) = ({\color{green}{q\Delta t}}) ^{-1} \mathcal{E}_{\color{green}{q}}^- 1_{y>ln(H)} \mathcal{E}_{\color{green}{q}}^+ f_{n+1}(y)$$

Получить хотим $f_0(y)$, там для каждого y и ответ лежит

Зелёные - константы. Определим их.

In [16]:
q = 1.0/delta_t + r
factor = (q*delta_t)**(-1)

# Зададим начальные уловия
f_N = array([G(y_element, H, K) for y_element in y_space]) # = g(x) i.e. payoff func

В уравнении выше есть нигде пока не определённные функции (на самом деле, операторы)
$$ \mathcal{E}^\pm_q f(x) = (2\pi)^{-1} \int_{-\infty}^{+\infty} e^{ix\xi} \phi^\pm_q (\xi) \hat{f}(\xi) d\xi $$

А в них - пока не определённые $\phi_q^\pm$

$\phi^-_q(\xi)$ будут иметь вид:

$$ \phi^-_q(\xi) = exp[-i\xi F^-(0) - \xi^2 \hat{F}^-(\xi)] $$
$$ F^-(x) = 1_{[0,+\infty)}(x)(2\pi)^{-1} \int_{-\infty}^{+\infty} e^{ix(\upsilon+i\omega_+)} \frac{ln(q+\psi(\upsilon+i\omega_+))}{(\upsilon+i\omega_+)^2} d(\upsilon) = 1_{[0,+\infty)}(x)\frac{e^{i\omega_+}\cdot 1}{(2\pi)} \int_{-\infty}^{+\infty} e^{ix(\upsilon)} \frac{ln(q+\psi(\upsilon+i\omega_+))}{(\upsilon+i\omega_+)^2} d(\upsilon) $$
$$ \hat{F}^-(\xi) = \int_{-\infty}^{+\infty} e^{-ix\xi} F^-(x) dx $$

In [ ]:
# преобразование Фурье должно допускать вычисление интеграла по \xi
# но поскольку мы имеем дело с ПДО, нужно ещё знать, с какими именно
xi_space = fft.fftfreq(M, d=d)
upsilon_space = fft.fftfreq(M, d=d)

omega_plus = 1


def integrand(upsilon):
    return log(q + psi(upsilon + i*omega_plus)) / (upsilon + i*omega_plus)^2

integrand_vals = [integrand(upsilon) for upsilon in upsilon_space]

# factor functions
def F_capital(x_array):
    exp(i*omega_plus) * fft.ifft(integrand_vals)
    
    

phi_minus_array = array([-beta_minus/(-beta_minus + i*2*pi*xi) for xi in xi_space])

# main cycle (iterating over time)
f_n = array(f_N)
#plt.plot(original_prices_array, f_n)
for i in range(N):
   f_n_plus_1 = factor * fft.ifft(phi_minus_array *
       fft.fft(indicator(fft.ifft(phi_plus_array * fft.fft(f_n)))))
   f_n = f_n_plus_1

plt.plot(original_prices_array, f_n_plus_1)

plt.savefig("out.png")
plt.show()